#  Proyecto de análisis de datos en Python. Análisis de la rotación de plantilla <br>

Dataset disponible en Kaggle: __[HRAnalytics](https://www.kaggle.com/ludobenistant/hr-analytics )__ <br>
Datos acerca del personal de una empresa, el objetivo es averiguar que empleados tienen mayor probabilidad de abandonarla.<br>

## Paso 1

Preparamos el cuaderno de trabajo importando las librerías necesarias y cargamos el archivo.

In [ ]:
# Importación de las librerías de análisis y carga del archivo
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as stats
import matplotlib.pyplot as plt
import matplotlib as matplot
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
filename = '../input/HR_comma_sep.csv'
df = pd.read_csv(filename)

## Paso 2: Comprobación y preparación de los datos

In [ ]:
print(df)

In [ ]:
# Número de filas
print(len(df))

In [ ]:
#Columnas
print(df.columns)
len(df.columns)

#Informacion de las columnas
df.info()

In [ ]:
# Ya que hay dos columnas categóricas, observamos los valores que pueden tomar.
print(df.sales.unique())
print(df.salary.unique())

In [ ]:
#Encabezados
df.head()

In [ ]:
# Comprobar si hay algún valor vacío en la tabla.
df.isnull().any()

In [ ]:
# Poner un nombre más apropiado a la columna 'sales':
df = df.rename(columns={'sales': 'Department'})

In [ ]:
# Convertimos las columnas "salary" y "Department" en categorías.
df.salary = df.salary.astype('category')
df.Department = df.Department.astype('category')
df.info()

## Paso 3: Exploración de los datos

In [ ]:
# Estadísticos descriptivos de los datos
df.describe()

In [ ]:
# Cantidad de personas que abandonaron la empresa y porcentaje sobre el total de empleados
print(df.left.value_counts())
print(int(df[df['left']==1].size)/df.size *100, '% de empleados abandonaron la empresa')

In [ ]:
#Comparación de medias entre empleados que abandonaron la empresa y los que siguen
df.groupby('left').mean()

In [ ]:
#Matriz de correlación
corr = df.corr()
corr

In [ ]:
#Mapa de calor
sns.heatmap(corr)
plt.show()

Información importante hasta ahora:<br>
- El porcentaje de empleados que abandona la empresa es de __23.8%__

Existe una __correlación positiva__ importante entre:
- Número de proyectos y última evaluación : __0.349__
- Número de proyectos y media de horas mensuales: __0.417__
- media de horas mensuales y última evaluación: __0.339__

Existe una __correlación negativa__ importante entre:
- Satisfacción y abandono de la empresa: __-0.388__

In [ ]:
# Vamos a comprobar como afecta la banda salarial
by_salary = df.groupby('salary').mean()
print(df['salary'].value_counts())
plt.figure(figsize=(6, 6))
sns.countplot(data=df, x="salary", hue="left")
by_salary

In [ ]:
sns.barplot(data=df, x="salary", y="left")

A priori, parecer __haber relación__ entre la banda salarial y la posibilidad de que un empleado abandone la empresa, aquellos en la banda salarial más alta tienen una menor rotación de plantilla.

In [ ]:
# Comprobamos las diferencias entre departamentos
by_department = df.groupby('Department').mean()
print(df['Department'].value_counts())
plt.figure(figsize=(8, 8))
sns.countplot(data=df, x="Department", hue="left")
plt.xticks(rotation= -45)
by_department

In [ ]:
plt.xticks(rotation= -45)
sns.barplot(data=df, x="Department", y="left")

----

Los departamentos con mayor rotación proporcional son: __Recursos Humanos__ y __Contabilidad__<br>
Los departamentos con menor rotación son: __Management__ e __Investigación y Desarrollo__

### Otras visualizaciones

In [ ]:
# Primera columna: distribución de los empleados en función de su nivel de satisfacción, última evaluación y media de horas mensuales.
# Segunda columna: Comparación entre los que abandonan la empresa y los que no.
f, ax = plt.subplots(3, 2, figsize=(30,15))
sns.distplot(df["satisfaction_level"], ax = ax[0,0])
sns.kdeplot(df.loc[(df['left'] == 1),'satisfaction_level'], shade=True, label='turnover', ax = ax[0,1])
sns.kdeplot(df.loc[(df['left'] == 0),'satisfaction_level'], shade=True, label='no turnover', ax = ax[0,1])
sns.distplot(df["last_evaluation"], color ='green', ax = ax[1,0])
sns.kdeplot(df.loc[(df['left'] == 1),'last_evaluation'], shade=True, label='turnover', ax = ax[1,1])
sns.kdeplot(df.loc[(df['left'] == 0),'last_evaluation'], shade=True, label='no turnover', ax = ax[1,1])
sns.distplot(df["average_montly_hours"], color ='red', ax = ax[2,0])
sns.kdeplot(df.loc[(df['left'] == 1),'average_montly_hours'], shade=True, label='turnover', ax = ax[2,1])
sns.kdeplot(df.loc[(df['left'] == 0),'average_montly_hours'], shade=True, label='no turnover', ax = ax[2,1])

- Parece ser que los empleados con menores niveles de __satisfacción__ (entre __0.2 y 0.4__) tienden más a abandonar la empresa; además aquellos con una satisfacción muy alta (__>0.8__) también tienen mayor probabilidad de abandonar la empresa.
- Respecto a su última __evaluación__, tanto aquellos con una evaluación baja (entre __0.4__ y __0.6__) como muy alta (__>0.8__) tienen mayores posibilidades de abandonar la empresa.
- Por último, tanto aquellos trabajadores con __menos de 160 horas mensuales__ como los que trabajan __más de 260 horas__ tienen mayor probabilidad de abandonar la empresa.